# Intro

- https://github.com/GoogleCloudDataproc/cloud-dataproc/tree/master/notebooks
- https://www.cloudskillsboost.google/focuses/3390?locale=fr&parent=catalog
- https://www.cloudskillsboost.google/focuses/5834?locale=fr&parent=catalog
- https://cloud.google.com/dataproc/docs/tutorials/gcs-connector-spark-tutorial?hl=fr#scala

spark:
- https://sparkbyexamples.com/spark/spark-sql-distinct-multiple-columns/
- https://mungingdata.com/apache-spark/dates-times/
- https://mungingdata.com/apache-spark/just-enough-scala/

In [1]:
println("Scala language: "+util.Properties.versionString)

Scala language: version 2.12.18


In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.expressions.Window
import java.time.LocalTime
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._

// import spark.implicits._

import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

import spark.sqlContext.implicits._

// spark.sparkContext.version
spark.version

spark = org.apache.spark.sql.SparkSession@104dfb99


3.3.2

---

# Flights

In [3]:
val filePath = "gs://dataset-flight/Flights/201201.csv"
val df_flights = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(filePath)

df_flights.show(3)

+-------------------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+----+
|            FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY|_c12|
+-------------------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+----+
|2012-01-01 00:00:00|                20366|             4426|            15370|          12266|         845|          0.0|      0.0|     0.0|            99.0|         null|     null|null|
|2012-01-01 00:00:00|                20366|             4427|            12266|          15370|         858|          0.0|      0.0|     0.0|            88.0|         null|     null|null|
|2012-01-01 00:00:00|                20366|             4427

filePath = gs://dataset-flight/Flights/201201.csv
df_flights = [FL_DATE: timestamp, OP_CARRIER_AIRLINE_ID: int ... 11 more fields]


[FL_DATE: timestamp, OP_CARRIER_AIRLINE_ID: int ... 11 more fields]

In [4]:
val nbRows = df_flights.count()
print(nbRows)

df_flights.printSchema

486133root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- _c12: string (nullable = true)



nbRows = 486133


486133

In [5]:
// https://sparkbyexamples.com/spark/spark-find-count-of-null-empty-string-values/
// print(df_flights.filter(col("_c12").isNull || col("_c12") === "").count() / nbRows)

for (columnName <- df_flights.columns) println(columnName, df_flights.filter(col(columnName).isNull || col(columnName) === "").count() / nbRows)

(FL_DATE,0)
(OP_CARRIER_AIRLINE_ID,0)
(OP_CARRIER_FL_NUM,0)
(ORIGIN_AIRPORT_ID,0)
(DEST_AIRPORT_ID,0)
(CRS_DEP_TIME,0)
(ARR_DELAY_NEW,0)
(CANCELLED,0)
(DIVERTED,0)
(CRS_ELAPSED_TIME,0)
(WEATHER_DELAY,0)
(NAS_DELAY,0)
(_c12,1)


In [10]:
for (columnName <- df_flights.columns) println("Nb nulls " + columnName + ": " + df_flights.filter(col(columnName).isNull || col(columnName) === "").count())

Nb nulls FL_DATE: 0
Nb nulls OP_CARRIER_AIRLINE_ID: 0
Nb nulls OP_CARRIER_FL_NUM: 0
Nb nulls ORIGIN_AIRPORT_ID: 0
Nb nulls DEST_AIRPORT_ID: 0
Nb nulls CRS_DEP_TIME: 0
Nb nulls ARR_DELAY_NEW: 8108
Nb nulls CANCELLED: 0
Nb nulls DIVERTED: 0
Nb nulls CRS_ELAPSED_TIME: 0
Nb nulls WEATHER_DELAY: 415225
Nb nulls NAS_DELAY: 415225
Nb nulls _c12: 486133


In [8]:
val flightColumns = List("FL_DATE", "OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID")

// df_flights.select(flightColumns.map(m=>col(m)):_*).describe().show()
df_flights.select(flightColumns.map(m=>col(m)):_*)
    .summary("count", "approx_count_distinct", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

flightColumns = List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID)


+--------------------+---------------------+------------------+------------------+------------------+
|             summary|OP_CARRIER_AIRLINE_ID| OP_CARRIER_FL_NUM| ORIGIN_AIRPORT_ID|   DEST_AIRPORT_ID|
+--------------------+---------------------+------------------+------------------+------------------+
|               count|               486133|            486133|            486133|            486133|
|approx_count_dist...|                   15|              5960|               288|               288|
|                mean|   20023.865104816996|2270.0695550394644|12682.037222323932|12682.126323454693|
|              stddev|    394.3032043391765|1900.9371190193674|1523.4647462359062|1523.3517870660357|
|                 min|                19393|                 1|             10135|             10135|
|                 25%|                19790|               680|             11292|             11292|
|                 50%|                19977|              1646|             12889|

List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID)

In [16]:
println("Distinct count date/carrier_id/fl_num: "+df_flights.select("FL_DATE", "OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM").distinct().count())
println("Distinct count date/all iDs & origin : "+df_flights.select("FL_DATE", "OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM", "ORIGIN_AIRPORT_ID").distinct().count())
println("Distinct count date/all iDs & numbers: "+df_flights.select("FL_DATE", "OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID").distinct().count())
println("Distinct count all columns           : "+df_flights.distinct().count())


Distinct count date/carrier_id/fl_num: 355439
Distinct count date/all iDs & origin : 486133
Distinct count date/all iDs & numbers: 486133
Distinct count all columns           : 486133


identification de la clé de jointure

Le meme jour:
- Des vols de différentes company peuvent avoir le meme numéro le meme jour.  
- Des vols de la meme company peuvent avoir le meme numéro mais pas le meme aéroport de départ (heureusement !)

donc les champs suivants doivent servir de clés: "FL_DATE", "OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM", "ORIGIN_AIRPORT_ID"

In [9]:
val otherColumns = List("CRS_DEP_TIME", "ARR_DELAY_NEW", "CANCELLED", "DIVERTED", "CRS_ELAPSED_TIME", "WEATHER_DELAY", "NAS_DELAY")

df_flights.select(otherColumns.map(m=>col(m)):_*)
    .summary("count", "approx_count_distinct", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

+--------------------+-----------------+------------------+--------------------+--------------------+-----------------+------------------+------------------+
|             summary|     CRS_DEP_TIME|     ARR_DELAY_NEW|           CANCELLED|            DIVERTED| CRS_ELAPSED_TIME|     WEATHER_DELAY|         NAS_DELAY|
+--------------------+-----------------+------------------+--------------------+--------------------+-----------------+------------------+------------------+
|               count|           486133|            478025|              486133|              486133|           486133|             70908|             70908|
|approx_count_dist...|             1133|               561|                   2|                   2|              432|               270|               269|
|                mean|1325.909621441046| 9.076759583703781|0.014613284841802552|0.002065278432034...|135.4468242230007|  3.03854290066001|13.882961019913127|
|              stddev|461.3563454701226|29.373017386

otherColumns = List(CRS_DEP_TIME, ARR_DELAY_NEW, CANCELLED, DIVERTED, CRS_ELAPSED_TIME, WEATHER_DELAY, NAS_DELAY)


List(CRS_DEP_TIME, ARR_DELAY_NEW, CANCELLED, DIVERTED, CRS_ELAPSED_TIME, WEATHER_DELAY, NAS_DELAY)

In [10]:
(2359/60, 1450.0/60, 1865.0/60 ,1179.0/60, 1007.0/60)

(39,24.166666666666668,31.083333333333332,19.65,16.783333333333335)

---
# Airport / WBAN / timezone

In [18]:
val airportsPath = "gs://dataset-flight/wban_airport_timezone.csv"
val df_airports = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(airportsPath)

df_airports.show(5)

+---------+-----+--------+
|AirportID| WBAN|TimeZone|
+---------+-----+--------+
|    10685|54831|      -6|
|    14871|24232|      -8|
|    10620|24033|      -7|
|    14747|24233|      -8|
|    11252|12834|      -5|
+---------+-----+--------+
only showing top 5 rows



airportsPath = gs://dataset-flight/wban_airport_timezone.csv
df_airports = [AirportID: int, WBAN: int ... 1 more field]


[AirportID: int, WBAN: int ... 1 more field]

In [19]:
df_airports.printSchema()

root
 |-- AirportID: integer (nullable = true)
 |-- WBAN: integer (nullable = true)
 |-- TimeZone: integer (nullable = true)



In [23]:
df_airports.summary("count", "count_distinct", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

+--------------+------------------+------------------+------------------+
|       summary|         AirportID|              WBAN|          TimeZone|
+--------------+------------------+------------------+------------------+
|         count|               305|               305|               305|
|count_distinct|               305|               304|                 7|
|          mean|12715.895081967214|30657.439344262297|-6.331147540983607|
|        stddev|1651.8968489231736|30096.398136759515|1.3123328193785093|
|           min|             10136|              3011|               -10|
|           25%|             11278|             13893|                -7|
|           50%|             12436|             14958|                -6|
|           75%|             14122|             24283|                -5|
|           max|             16440|             94910|                -4|
+--------------+------------------+------------------+------------------+



---

# Weather

In [3]:
val weatherPath = "gs://dataset-flight/Weather/201201hourly.txt"
val df_weather = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(weatherPath)

weatherPath = gs://dataset-flight/Weather/201201hourly.txt
df_weather = [WBAN: int, Date: int ... 42 more fields]


[WBAN: int, Date: int ... 42 more fields]

In [4]:
df_weather.printSchema()

root
 |-- WBAN: integer (nullable = true)
 |-- Date: integer (nullable = true)
 |-- Time: integer (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: string (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: string (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: string (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: string (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: string (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: string (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: string (nullable = true)
 |-- DewPointCelsiusFlag: str

In [5]:
df_weather.select("WBAN", "Date", "Time", "StationType").show(3)

+----+--------+----+-----------+
|WBAN|    Date|Time|StationType|
+----+--------+----+-----------+
|3011|20120101|  15|          0|
|3011|20120101|  35|          0|
|3011|20120101|  55|          0|
+----+--------+----+-----------+
only showing top 3 rows



In [6]:
df_weather.select("WBAN", "Date", "Time", "StationType").summary("count", "approx_count_distinct", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

+--------------------+------------------+-------------------+------------------+-----------------+
|             summary|              WBAN|               Date|              Time|      StationType|
+--------------------+------------------+-------------------+------------------+-----------------+
|               count|           4192912|            4192912|           4192912|          4192912|
|approx_count_dist...|              2040|                 31|              1494|                6|
|                mean|39532.467004077356|2.012011601765694E7|1176.1133064562289|8.856116465120184|
|              stddev|31980.126776193927|  8.872467923216242| 693.2235883206993|6.782955134108237|
|                 min|              3011|           20120101|                 0|                0|
|                 25%|              4951|           20120108|               555|                0|
|                 50%|             26648|           20120116|              1155|               12|
|         

a ne pas relancer car prend du temps:

In [9]:
for (columnName <- df_weather.columns) println("% nulls " + columnName + ": " + df_weather.filter(col(columnName).isNull || col(columnName) === "").count() / df_weather.count() * 100)

% nulls WBAN: 0
% nulls Date: 0
% nulls Time: 0
% nulls StationType: 0
% nulls SkyCondition: 0
% nulls SkyConditionFlag: 0
% nulls Visibility: 0
% nulls VisibilityFlag: 0
% nulls WeatherType: 0
% nulls WeatherTypeFlag: 0
% nulls DryBulbFarenheit: 0
% nulls DryBulbFarenheitFlag: 0
% nulls DryBulbCelsius: 0
% nulls DryBulbCelsiusFlag: 0
% nulls WetBulbFarenheit: 0
% nulls WetBulbFarenheitFlag: 0
% nulls WetBulbCelsius: 0
% nulls WetBulbCelsiusFlag: 0
% nulls DewPointFarenheit: 0
% nulls DewPointFarenheitFlag: 0
% nulls DewPointCelsius: 0
% nulls DewPointCelsiusFlag: 0
% nulls RelativeHumidity: 0
% nulls RelativeHumidityFlag: 0
% nulls WindSpeed: 0
% nulls WindSpeedFlag: 0
% nulls WindDirection: 0
% nulls WindDirectionFlag: 0
% nulls ValueForWindCharacter: 0
% nulls ValueForWindCharacterFlag: 0
% nulls StationPressure: 0
% nulls StationPressureFlag: 0
% nulls PressureTendency: 0
% nulls PressureTendencyFlag: 0
% nulls PressureChange: 0
% nulls PressureChangeFlag: 0
% nulls SeaLevelPressur

In [8]:
df_weather.columns

Array(WBAN, Date, Time, StationType, SkyCondition, SkyConditionFlag, Visibility, VisibilityFlag, WeatherType, WeatherTypeFlag, DryBulbFarenheit, DryBulbFarenheitFlag, DryBulbCelsius, DryBulbCelsiusFlag, WetBulbFarenheit, WetBulbFarenheitFlag, WetBulbCelsius, WetBulbCelsiusFlag, DewPointFarenheit, DewPointFarenheitFlag, DewPointCelsius, DewPointCelsiusFlag, RelativeHumidity, RelativeHumidityFlag, WindSpeed, WindSpeedFlag, WindDirection, WindDirectionFlag, ValueForWindCharacter, ValueForWindCharacterFlag, StationPressure, StationPressureFlag, PressureTendency, PressureTendencyFlag, PressureChange, PressureChangeFlag, SeaLevelPressure, SeaLevelPressureFlag, RecordType, RecordTypeFlag, HourlyPrecip, HourlyPrecipFlag, Altimeter, AltimeterFlag)

https://www.ncei.noaa.gov/resources/metadata
Quality Controlled Local Climatological Data (QCLCD

In [15]:
df_weather.select(df_weather.columns.take(12).map(col): _*).show(10)

+----+--------+----+-----------+------------+----------------+----------+--------------+-----------+---------------+----------------+--------------------+
|WBAN|    Date|Time|StationType|SkyCondition|SkyConditionFlag|Visibility|VisibilityFlag|WeatherType|WeatherTypeFlag|DryBulbFarenheit|DryBulbFarenheitFlag|
+----+--------+----+-----------+------------+----------------+----------+--------------+-----------+---------------+----------------+--------------------+
|3011|20120101|  15|          0|         CLR|                |     10.00|              |           |               |              23|                    |
|3011|20120101|  35|          0|         CLR|                |     10.00|              |           |               |              21|                    |
|3011|20120101|  55|          0|         CLR|                |     10.00|              |           |               |              21|                    |
|3011|20120101| 115|          0|         CLR|                |     10.

In [31]:
for (columnName <- df_weather.columns.filter(x => x.endsWith("Flag"))) {
  println("10 of the disctinct values for " + columnName + ": " + df_weather.select(columnName).distinct().take(10).mkString(", "))
}

10 of the disctinct values for SkyConditionFlag: [ ], [s]
10 of the disctinct values for VisibilityFlag: [ ], [s]
10 of the disctinct values for WeatherTypeFlag: [ ], [s]
10 of the disctinct values for DryBulbFarenheitFlag: [ ]
10 of the disctinct values for DryBulbCelsiusFlag: [ ], [s]
10 of the disctinct values for WetBulbFarenheitFlag: [ ]
10 of the disctinct values for WetBulbCelsiusFlag: [ ]
10 of the disctinct values for DewPointFarenheitFlag: [ ]
10 of the disctinct values for DewPointCelsiusFlag: [ ], [s]
10 of the disctinct values for RelativeHumidityFlag: [ ]
10 of the disctinct values for WindSpeedFlag: [ ], [s]
10 of the disctinct values for WindDirectionFlag: [ ]
10 of the disctinct values for ValueForWindCharacterFlag: [ ], [s]
10 of the disctinct values for StationPressureFlag: [ ]
10 of the disctinct values for PressureTendencyFlag: [ ]
10 of the disctinct values for PressureChangeFlag: [ ]
10 of the disctinct values for SeaLevelPressureFlag: [ ], [s]
10 of the disctinc

In [32]:
for (columnName <- df_weather.columns.filter(x => !x.endsWith("Flag"))) {
  println("10 of the disctinct values for " + columnName + ": " + df_weather.select(columnName).distinct().take(10).mkString(", "))
}

10 of the disctinct values for WBAN: [3749], [3918], [3997], [4935], [3179], [4929], [3704], [3761], [3089], [3098]
10 of the disctinct values for Date: [20120122], [20120127], [20120109], [20120103], [20120112], [20120123], [20120114], [20120108], [20120129], [20120131]
10 of the disctinct values for Time: [148], [1342], [2122], [2142], [1959], [1238], [833], [1645], [1829], [2235]
10 of the disctinct values for StationType: [12], [6], [5], [15], [11], [0]
10 of the disctinct values for SkyCondition: [BKN050], [SCT050 BKN055], [BKN120], [SCT013 BKN026 OVC031], [SCT012 OVC025], [BKN039 BKN075], [BKN028 BKN036], [SCT010 OVC020], [SCT003 SCT028 OVC037], [SCT022 BKN050]
10 of the disctinct values for Visibility: [ 8.00], [ 2.00], [90.00], [15.00], [ 5.00], [17.00], [80.00], [30.00], [13.00], [ 1.25]
10 of the disctinct values for WeatherType: [DZ], [-RASN BR], [UP FZFG], [+RA FG], [-SHRA BR], [VCTS -FZRA BR], [-RA SQ], [TSDZ], [-SNPL], [-FZRA SNPL BR]
10 of the disctinct values for DryBul

une grande partie des features sont en string à cause de l'espace à la place des NULL ! recalcul du % du null par colonne:

In [33]:
val nbRows = df_weather.count()
for (columnName <- df_weather.columns) {
    println("% nulls " + columnName + ": " + df_weather.filter(col(columnName).isNull || col(columnName) === "" || col(columnName) === " " || col(columnName) === "  ").count() * 100 / nbRows)
}

% nulls WBAN: 0
% nulls Date: 0
% nulls Time: 0
% nulls StationType: 0
% nulls SkyCondition: 0
% nulls SkyConditionFlag: 99
% nulls Visibility: 0
% nulls VisibilityFlag: 99
% nulls WeatherType: 88
% nulls WeatherTypeFlag: 99
% nulls DryBulbFarenheit: 0
% nulls DryBulbFarenheitFlag: 100
% nulls DryBulbCelsius: 0
% nulls DryBulbCelsiusFlag: 99
% nulls WetBulbFarenheit: 0
% nulls WetBulbFarenheitFlag: 100
% nulls WetBulbCelsius: 0
% nulls WetBulbCelsiusFlag: 100
% nulls DewPointFarenheit: 0
% nulls DewPointFarenheitFlag: 100
% nulls DewPointCelsius: 0
% nulls DewPointCelsiusFlag: 99
% nulls RelativeHumidity: 0
% nulls RelativeHumidityFlag: 100
% nulls WindSpeed: 0
% nulls WindSpeedFlag: 99
% nulls WindDirection: 0
% nulls WindDirectionFlag: 100
% nulls ValueForWindCharacter: 88
% nulls ValueForWindCharacterFlag: 99
% nulls StationPressure: 0
% nulls StationPressureFlag: 100
% nulls PressureTendency: 93
% nulls PressureTendencyFlag: 100
% nulls PressureChange: 93
% nulls PressureChangeFlag

nbRows = 4192912


4192912